# TAMU Datathon - Bloomberg Challenge (post-competition work)

I wanted to revisit this project, and see it to completion.

Part 1 was guessing what 5 embeddings' original news articles were about (see other notebook).

Part 2 was building a general classifier for Bloomberg's embedding system.

This will have two phases:

A: A genre classifier for bodies of text, trained on a large set of articles. This will be used to generate genre labels for the set of ~1,000 embeddings we were originally given at the start of the contest.

B: A genre classifier for Bloomberg's embeddings. This will only be trained using the ~1,000 embeddings we were given as features, and the labels generated in Part A as targets.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np
import pandas as pd
import json

# PART 2: Classifier

## Phase A: Genre Classifier, Label Generation for Embedding Set

First, import the training sets (texts with genre labels) and embedding data (embeddings with texts).

Second, preprocess the data using TF-IDF vectors.

Third, train and cross-validate a supervised learning model for label generation.

Fourth, once the model is satisfactory, apply to the embedding data.

Fifth, export the embedding-label pairings (so I don't have to repeat this cell).

In [ ]:
cnn_samples0 = pd.read_csv('cnn_samples-54b19b96f3c0775b116bad527df8c7b5.csv')

# Wrangling the data from strings to NP arrays.
cnn_samples1 = np.fromstring((cnn_samples0.values[0,3]).replace('[','').replace(']',''), sep=',').reshape(1,512)

# Rebuilding the DataFrame after this, with headline as index.
for i in np.arange(1,np.shape(cnn_samples0)[0]):
    temp = np.fromstring((cnn_samples0.values[i,3]).replace('[','').replace(']',''), sep=',').reshape(1,512)
    cnn_samples1 = np.vstack([cnn_samples1, temp])
cnn_samples = pd.DataFrame(cnn_samples1, index = cnn_samples0['text'])

#Repeating the process for the challenge data.
gov_samples0 = pd.read_csv('federal_samples-a586d0681e005629453435bea5b173eb.csv')
gov_samples1 = np.fromstring((gov_samples0.values[0,3]).replace('[','').replace(']',''), sep=',').reshape(1,512)
for i in np.arange(1,np.shape(gov_samples0)[0]):
    temp = np.fromstring((gov_samples0.values[i,3]).replace('[','').replace(']',''), sep=',').reshape(1,512)
    gov_samples1 = np.vstack([gov_samples1, temp])
gov_samples = pd.DataFrame(gov_samples1, index = gov_samples0['text'])

#also need to merge the two DataFrames
cnngov_samples = pd.concat([cnn_samples, gov_samples], axis = 'rows')

#Repeating the process for the challenge data.
challenge0 = pd.read_csv('challenge-ddec63cf66ea88f128e3c21e457f393a.csv')
challenge1 = np.fromstring((challenge0.values[0,1]).replace('[','').replace(']',''), sep=',').reshape(1,512)
for i in np.arange(1,np.shape(challenge0)[0]):
    temp = np.fromstring((challenge0.values[i,1]).replace('[','').replace(']',''), sep=',').reshape(1,512)
    challenge1 = np.vstack([challenge1, temp])
challenge = pd.DataFrame(challenge1, index = challenge0['id'])


#Finally, getting the mystery 6th article.
with open('mystery.json') as file:
    mystery0 = json.load(file)['embedding']

mystery = pd.DataFrame(np.array(mystery0).reshape(1,512), index = ['mystery'], columns = np.arange(0,512)) #it's a dict

#also need to merge these two DataFrames
challenge = pd.concat([challenge, mystery], axis = 'rows')

#print(cnngov_samples.head()) #just double-checking
#print(challenge.head())

## Phase B: Genre Classifier for the Embeddings

First, import the CSV file generated by the previous cell.

Second, train and cross-validate a supervised learning model for label generation.

Third, once the model is satisfactory, predict the genres of the five challenge articles to test.

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.mixture import GaussianMixture

